In [2]:
# Python version
import sys
print('Python: {}'.format(sys.version))
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
import matplotlib.pyplot as plt
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
import pickle as pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler

Python: 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]
numpy: 1.14.0
matplotlib: 2.1.2
pandas: 0.22.0


In [4]:
def pickle_data(filename, data):

    fileObject = open('PickleData/'+filename,'wb') 
    # this writes the object a to the
    # file named 'testfile'
    pickle.dump(data,fileObject) 

    # here we close the fileObject
    fileObject.close()
    
def load_pickle_data(filename):
    # we open the file for reading
    fileObject = open('PickleData/'+filename,'rb')  
    # load the object from the file into var b
    return pickle.load(fileObject)

In [5]:
df = pd.read_csv('Data/test.csv')

## Data Processing: Non-Binary Dataset

In [6]:
# Drop unrequired features
#For testing set
df = df.drop(['bidid','userid','IP','domain','url','urlid','slotid','slotprice','creative','keypage'],axis=1)


# Split user agent into two features, platform and browser
df['platform'] = df['useragent'].map(lambda x: str(x).split('_')[0])   
df['browser'] = df['useragent'].map(lambda x: str(x).split('_')[1])

#Encode as integer values
df.loc[df['slotvisibility'] == 'FirstView', 'slotvisibility'] = 1
df.loc[df['slotvisibility'] == 'SecondView', 'slotvisibility'] = 2
df.loc[df['slotvisibility'] == 'ThirdView', 'slotvisibility'] = 3
df.loc[df['slotvisibility'] == 'FourthView', 'slotvisibility'] = 4
df.loc[df['slotvisibility'] == 'FifthView', 'slotvisibility'] = 5
df.loc[df['slotvisibility'] == 'OtherView', 'slotvisibility'] = 7
df.loc[df['slotvisibility'] == 'Na', 'slotvisibility'] = 8
df.loc[df['slotvisibility'] == '255', 'slotvisibility'] = 6

#Replace nulls
df.loc[df['adexchange'].isna(), 'adexchange'] = 0

#Replace nulls as strings
df.loc[df['slotformat'] == 'Na', 'slotformat'] = 0

#One-hot encode features
df = df.join(df['usertag'].str.get_dummies(','))

df = df.drop(['useragent'],axis=1)

In [7]:
#One-hot encode features
df = df.join(pd.get_dummies(df['platform'],prefix='platform'))
df = df.join(pd.get_dummies(df['browser'],prefix='browser'))

In [8]:
df = df.drop(['usertag', 'platform','browser'], axis=1)

In [13]:
pickle_data('train_normal_clean', df)

## Option: Under Sample

In [27]:
# Apply the random under-sampling
rus = RandomUnderSampler(return_indices=True)
X_resampled, y_resampled, idx_resampled = rus.fit_sample(df.drop(['click'],axis=1), df['click'])
# X_res_vis = pca.transform(X_resampled)

In [30]:
pickle_data('normal_clean_y_val_undersampled', y_resampled)
pickle_data('normal_clean_x_val_undersampled', X_resampled)